# 1. Import Dependencies

In [26]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

## Dataset
This specific dataset seperates hand gestures into 4 different classes.

- thumbsup
- thumbsdown
- thankyou
- livelong

The information about each hand gesture is contained into 42 features.

In [27]:
CSV_COLUMN_NAMES = np.arange(43)
CSV_COLUMN_NAMES = [str(item) for item in CSV_COLUMN_NAMES]
CSV_COLUMN_NAMES[42] = "Species"
SPECIES = ['stop', 'indexup', 'twofingerup', 'punch', 'thumbsup']
CSV_PATH = os.path.join("src", "dataHandGesture", "file_0.csv")

In [28]:
df = pd.read_csv(CSV_PATH, names=CSV_COLUMN_NAMES, header=0)
train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)

In [29]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,Species
236,496,329,473,305,453,286,442,270,436,259,...,291,473,302,479,275,481,284,0,0,5
237,499,246,472,239,454,230,450,228,453,230,...,217,516,187,512,214,511,223,0,0,5
342,234,432,227,461,201,489,177,500,159,501,...,438,156,408,141,424,163,427,0,0,8
142,401,454,391,445,390,431,397,423,407,414,...,460,430,447,430,455,422,468,0,0,3
58,205,403,182,378,172,345,175,318,180,298,...,290,212,356,214,337,213,329,0,0,1


In [30]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() # the species column is now gone

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
236,496,329,473,305,453,286,442,270,436,259,...,466,291,473,302,479,275,481,284,0,0
237,499,246,472,239,454,230,450,228,453,230,...,493,217,516,187,512,214,511,223,0,0
342,234,432,227,461,201,489,177,500,159,501,...,183,438,156,408,141,424,163,427,0,0
142,401,454,391,445,390,431,397,423,407,414,...,414,460,430,447,430,455,422,468,0,0
58,205,403,182,378,172,345,175,318,180,298,...,188,290,212,356,214,337,213,329,0,0


In [31]:
train.shape  # we have 120 entires with 4 features

(287, 42)

## Input Function
Remember that nasty input function we created earlier. Well we need to make another one here! Fortunatly for us this one is a little easier to digest.

In [32]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [33]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='8', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='9', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='10', shape=(1,), default_value=None, dty

# 2. Building the Model
And now we are ready to choose a model. For classification tasks there are variety of different estimators/models that we can pick from. Some options are listed below.

- DNNClassifier (Deep Neural Network)
- LinearClassifier

We can choose either model but the DNN seems to be the best choice. This is because we may not be able to find a linear coorespondence in our data.

In [34]:
MODEL_PATH = os.path.join("Tensorflow", "workspace", "models", "my_hand_gesture_model")

In [35]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=len(SPECIES),
    model_dir= MODEL_PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'Tensorflow\\workspace\\models\\my_hand_gesture_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training

In [36]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 372.0624, step = 0
INFO:tensorflow:global_step/sec: 58.5447
INFO:tensorflow:loss = 72.55154, step = 100 (1.709 sec)
INFO:tensorflow:global_step/sec: 86.2763
INFO:tensorflow:loss = 52.0984, step = 200 (1.159 sec)
INFO:tensorflow:global_step/sec: 102.664
INFO:tensorflow:loss = 30.944654, step = 300 (0.973 sec)
INFO:tensorflow:global_step/sec: 112.733
INFO:tensorflow:loss = 30.210457, step = 400 (0.887 sec)
INFO:tensorflow:global_step/sec: 137.923
INFO:tensorflow:loss = 22.54586, s

## Export Model

In [37]:
# To export
feature_spec = tf.feature_column.make_parse_example_spec(my_feature_columns);
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec);
export_path = classifier.export_saved_model(MODEL_PATH, export_input_fn, as_text=True);

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: Tensorflow\workspace\models\my_hand_gesture_model\temp-1636125226\saved_model.pbtxt


## Load Model

In [38]:
# Loading the estimator
predict_fn = tf.saved_model.load(export_path).signatures['predict']
predict_fn

<ConcreteFunction pruned(examples) at 0x3C3FE3D0>

In [39]:
# Convert input data into serialized Example strings.
examples = []
for index, row in test.iterrows():
    feature = {}
    for col, value in row.iteritems():
        feature[col] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    example = tf.train.Example(
        features=tf.train.Features(
            feature=feature
        )
    )
    examples.append(example.SerializeToString())

# Convert from list to tensor
examples = tf.constant(examples)

print(test)
print(feature)

       0    1    2    3    4    5    6    7    8    9  ...   32   33   34  \
0    425  384  385  366  357  333  343  304  328  281  ...  437  201  461   
1    542  317  498  308  465  277  445  247  425  224  ...  542  123  576   
3    391  300  358  294  331  273  314  254  297  238  ...  389  154  414   
6    309  281  269  293  238  284  214  268  194  253  ...  271  133  323   
7    263  473  227  471  201  453  183  435  163  420  ...  261  320  288   
..   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
334  485  382  459  427  421  464  391  486  378  507  ...  405  387  380   
345  435  329  413  358  382  377  360  390  347  405  ...  376  328  360   
347  423  384  407  414  380  436  360  447  346  458  ...  369  395  345   
356  383  322  366  355  342  382  322  401  316  417  ...  321  330  306   
358  389  304  382  329  367  348  354  361  346  372  ...  358  299  341   

      35   36   37   38   39  40  41  
0    307  470  275  474  253   0   0

In [40]:
# make predictions of all testset
predictions = predict_fn(examples=examples)
print(predictions)

{'logits': <tf.Tensor: shape=(72, 9), dtype=float32, numpy=
array([[ 9.07929802e+00, -6.34903669e+00, -1.31726408e+00,
        -3.01138431e-01,  1.39099467e+00, -1.13401413e+01,
         8.83725262e+00, -1.07838030e+01,  5.95371485e+00],
       [-7.06798062e-02,  8.21789429e-02,  1.66113824e-02,
         1.25895077e-02, -6.44719899e-02,  6.35924861e-02,
        -7.48565644e-02,  4.34527127e-03,  5.05424999e-02],
       [ 3.46968842e+00, -2.40622854e+00, -4.99500275e-01,
        -1.08800136e-01,  4.98686522e-01, -4.34881401e+00,
         3.37347436e+00, -4.16987419e+00,  2.33463597e+00],
       [ 6.84930229e+00, -4.78164625e+00, -9.92177188e-01,
        -2.24677920e-01,  1.03627419e+00, -8.56086922e+00,
         6.66522932e+00, -8.15455914e+00,  4.51501751e+00],
       [ 1.01624136e+01, -7.11032248e+00, -1.47515965e+00,
        -3.38275522e-01,  1.56328332e+00, -1.26900415e+01,
         9.89221096e+00, -1.20608339e+01,  6.65249443e+00],
       [ 9.21703398e-01, -6.15334272e-01, -1.28057

In [41]:
# print results
for idx, resultPred in enumerate(predictions["class_ids"]):
    class_id = resultPred[0]
    probability = predictions['probabilities'][idx][class_id]
    print(probability)
    print(f"\tPrediction is {SPECIES[class_id]} {100 * probability :.2f}%")
    print(f"\tExpected: {SPECIES[test_y.iloc[idx]]}")

tf.Tensor(0.54659593, shape=(), dtype=float32)
	Prediction is stop 54.66%
	Expected: stop
tf.Tensor(0.12017216, shape=(), dtype=float32)
	Prediction is go 12.02%
	Expected: stop
tf.Tensor(0.42892262, shape=(), dtype=float32)
	Prediction is stop 42.89%
	Expected: stop
tf.Tensor(0.51733255, shape=(), dtype=float32)
	Prediction is stop 51.73%
	Expected: stop
tf.Tensor(0.55761725, shape=(), dtype=float32)
	Prediction is stop 55.76%
	Expected: stop
tf.Tensor(0.22713928, shape=(), dtype=float32)
	Prediction is stop 22.71%
	Expected: stop
tf.Tensor(0.12017216, shape=(), dtype=float32)
	Prediction is go 12.02%
	Expected: stop
tf.Tensor(0.12017216, shape=(), dtype=float32)
	Prediction is go 12.02%
	Expected: go
tf.Tensor(0.12017216, shape=(), dtype=float32)
	Prediction is go 12.02%
	Expected: go
tf.Tensor(0.12017216, shape=(), dtype=float32)
	Prediction is go 12.02%
	Expected: go
tf.Tensor(0.1813003, shape=(), dtype=float32)
	Prediction is stop 18.13%
	Expected: go
tf.Tensor(0.12017216, shape=(

## Evaluation

In [42]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-11-05T16:14:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.30002s
INFO:tensorflow:Finished evaluation at 2021-11-05-16:14:40
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.15277778, average_loss = 1.996594, global_step = 5000, loss = 1.996594
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000

Test set accuracy: 0.153



## Predictions
Now that we have a trained model it's time to use it to make predictions. I've written a little script below that allows you to type the features of a flower and see a prediction for its class.

In [43]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = CSV_COLUMN_NAMES[:-1]
predict = {}

for rowNum in range(5):
    expected = SPECIES[test_y.iloc[rowNum]]
    for idx, feature in enumerate(features):
      predict[feature] = [test.iloc[rowNum][idx]] #to predict the first row of test

    predictions = classifier.predict(input_fn=lambda: input_fn(predict))
    for pred_dict in predictions:
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print("\n")
        print('Prediction is "{}" ({:.1f}%)'.format(
            SPECIES[class_id], 100 * probability))
        print(f"Expected is {expected}")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "stop" (54.7%)
Expected is stop
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Prediction is "go" (12.0%)
Expected is stop
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from Tensorflow\workspace\models\my_hand_gesture_model\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


P